In [3]:
! pip3 install PyMuPDF

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
import fitz  # PyMuPDF
import requests
import os, json
pdf_url = "https://people.eecs.berkeley.edu/~vazirani/algorithms/chap0.pdf"
pdf_filename = "chap0.pdf"
response = requests.get(pdf_url)
if response.status_code == 200:
    with open(pdf_filename, 'wb') as pdf_file:
        pdf_file.write(response.content)
else:
    print("Failed to download the PDF file")
    exit()

def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

pdf_text = extract_text_from_pdf(pdf_filename)

In [5]:
pdf_json = {
    "content": pdf_text
}


In [6]:
json_filename = "chap0.json"
with open(json_filename, 'w') as json_file:
    json.dump(pdf_json, json_file, indent=4)


In [7]:
os.remove(pdf_filename)

print(f"PDF content has been converted to JSON and saved to {json_filename}")


PDF content has been converted to JSON and saved to chap0.json


In [52]:
with open('chap0.json', 'r') as file:
    data = json.load(file)

In [53]:
def split_text(text, max_length=21000):
    sentences = text.split('.')
    chunks = []
    chunk = ""
    for sentence in sentences:
        if len(chunk) + len(sentence) <= max_length:
            chunk += sentence + '.'
        else:
            chunks.append(chunk)
            chunk = sentence + '.'
    chunks.append(chunk)  # add the last chunk
    return chunks

chunks = split_text(data['content'])

In [54]:
import os
import json

from octoai.client import OctoAI
from octoai.text_gen import ChatMessage


# Get the OctoAI API token from the environment
OCTOAI_API_TOKEN = os.getenv("OCTOAI_API_TOKEN")
if not OCTOAI_API_TOKEN:
    raise ValueError("No OCTOAI_API_TOKEN found in environment variables")

# Initialize OctoAI client
client = OctoAI(api_key=OCTOAI_API_TOKEN)


def get_answer_from_chunk(chunk,question):
    completion = client.text_gen.create_chat_completion(
    model="qwen2-7b-instruct",
    messages=[
        ChatMessage(
            role="system",
            content="You are a professor understand the content of the PDF and explain it to a student.",
        ),
        ChatMessage(role="system", content=question),
        ChatMessage(role="user", content= chunk),
    ],
    max_tokens=512,
    )

    return json.dumps(completion.dict(), indent=2)

question = "What is the content of the PDF?"



In [64]:
i = 0
data = ""
for chunk in chunks:
    answer = get_answer_from_chunk(chunk, question)
    print(i)
    i += 1
    print("------------------------------------------------------------------------------------------------")
    data += answer
    
    



0
------------------------------------------------------------------------------------------------


In [68]:
# data_json = json.dumps(data, indent=2)

# # Print the JSON data
# print(data_json)

# # Parse the JSON string
# data = json.loads(data)

# # Extract the "content" key
# content = data['choices'][0]['message']['content']

# # Print the extracted content
# print(content)
print(data)
for json_string in data:
    data1 = json.loads(json_string)
    content = data1['choices'][0]['message']['content']
    print(content)

{
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "This PDF chapter primarily focuses on elucidating the importance of efficient algorithms in the context of the incredible proliferation and evolution of modern communication and technology. In particular, it outlines the pivotal role of algorithms through examples such as Gutenberg's printing press and Leonardo Fibonacci's sequence.\n\n### Fibonacci Numbers and Algorithms\n\nThe Fibonacci sequence is introduced as an example of a series of numbers where each number is the sum of the two preceding ones (0, 1, 1, 2, 3, 5, 8, 13, 21, 34, etc.). It's noted that these numbers are crucial in various fields due to their widespread attention and application.\n\n#### Recursive Algorithm (fib1)\n\nThis sequence can be simply generated using a recursive algorithm that mirrors its mathematical definition, utilizing a pseudocode-like notation:\n\n```plaintext\nfib

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)